# Web scraping stat Viego

In [173]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [174]:
url = "https://leagueoflegends.fandom.com/wiki/Viego/LoL"
url

'https://leagueoflegends.fandom.com/wiki/Viego/LoL'

*Statistic growth = base+ (perlvl) x (n-1) x (0.7025 + 0.0175 x (n-1))*

In [175]:
response = requests.get(url)
html = BeautifulSoup(response.text, "html.parser")

In [176]:
def findSpan(typeTag : str,  libTag : str):
    return html.find("span", {typeTag : libTag})

In [177]:
hp = findSpan("id","Health_Viego")
hp_lvl = findSpan("id","Health_Viego_lvl")
hp_list = [(float(hp.text) + float(hp_lvl.text) * (n-1) *(0.7025 +0.0175 * (n-1))) for n in range(1,19)]

In [178]:
armor =  findSpan("id","Armor_Viego")
armor_lvl = findSpan("id","Armor_Viego_lvl")
armor_list = [(float(armor.text) + float(armor_lvl.text) * (n-1) *(0.7025 +0.0175 * (n-1))) for n in range(1,19)]

In [179]:
rm = findSpan("id","MagicResist_Viego")
rm_lvl = findSpan("id","MagicResist_Viego_lvl")
rm_list = [(float(rm.text) + float(rm_lvl.text) * (n-1) *(0.7025 +0.0175 * (n-1))) for n in range(1,19)]

In [180]:
ad = findSpan("id","AttackDamage_Viego")
ad_lvl = findSpan("id","AttackDamage_Viego_lvl")
ad_list = [(float(ad.text) + float(ad_lvl.text) * (n-1) *(0.7025 +0.0175 * (n-1))) for n in range(1,19)]

In [181]:
attack_speed = html.findAll('div', {"style" : "width: calc(1 / 2 * 100%);"})
attack_speed = attack_speed[10]
attack_speed = float(re.sub(r'[^0-9.,]', '', attack_speed.text))
print(attack_speed)

0.658


In [182]:
bonus_as_lvl = html.findAll('div', {"style" : "width: calc(1 / 2 * 100%);"})
bonus_as_lvl = bonus_as_lvl[13]
bonus_as_lvl = float(re.sub(r'[^0-9.,]', '', bonus_as_lvl.text))
bonus_as_lvl /=100
print(bonus_as_lvl)
bonus_as_list = [ float(bonus_as_lvl) * (n-1) *(0.7025 +0.0175 * (n-1)) for n in range(1,19)]
print(bonus_as_list)

0.025
[0.0, 0.018, 0.036875000000000005, 0.05662500000000001, 0.07725, 0.09875, 0.12112500000000001, 0.14437500000000003, 0.1685, 0.19350000000000003, 0.21937500000000001, 0.24612500000000004, 0.27375000000000005, 0.30225, 0.33162500000000006, 0.36187500000000006, 0.393, 0.42500000000000004]


RATIO = 0.658
Formule total AS : *TOTAL = base + ratio*bonus

In [183]:
ratio = 0.658

In [184]:
as_list = [ attack_speed + ratio*bonus_as_list[n]  for n in range(0,18)]
print(as_list)

[0.658, 0.669844, 0.6822637500000001, 0.69525925, 0.7088305, 0.7229775, 0.7377002500000001, 0.75299875, 0.768873, 0.785323, 0.80234875, 0.8199502500000001, 0.8381275000000001, 0.8568805, 0.87620925, 0.89611375, 0.9165940000000001, 0.9376500000000001]


In [197]:
viegoStat = pd.DataFrame(index=range(1,19),columns=['hp','armor','magic_res','ad','as'])
viegoStat['hp'] = hp_list
viegoStat['armor'] = armor_list
viegoStat['magic_res'] = rm_list
viegoStat['ad'] = ad_list
viegoStat['as'] = bonus_as_list #Ici c'est le bonus d'attaque speed, il faut additionner les autres bonus
viegoStat['ViegoList'] = pViegoList 
viegoStat

,hp,armor,magic_res,ad,as,ViegoList
1,630.000,34.000,32.00000,57.0000,0.000000,0.02
2,708.480,37.744,33.47600,59.5200,0.018000,0.01425
3,790.775,41.670,35.02375,62.1625,0.036875,0.0002
4,876.885,45.778,36.64325,64.9275,0.056625,0.0
5,966.810,50.068,38.33450,67.8150,0.077250,0
6,1060.550,54.540,40.09750,70.8250,0.098750,0
7,1158.105,59.194,41.93225,73.9575,0.121125,0
8,1259.475,64.030,43.83875,77.2125,0.144375,0
9,1364.660,69.048,45.81700,80.5900,0.168500,0
10,1473.660,74.248,47.86700,84.0900,0.193500,0


In [186]:
pViego = html.find("div" ,{'class' : 'skill skill_innate' })
wViego = html.find("div", {"class" : "skill skill_w"})

In [203]:
pViegoList = []


try :
    pViegoList.append(str(float(re.sub(r'[^0-9.,]', '', pViego.find("span", {'style' : "color: #1F995C; white-space:normal"}).text))/100))
    print(pViegoList)
    
    pViegoList.append(str(float(re.sub(r'[^0-9.,]', '',pViego.find("span",{"style" : "color:orange; white-space:normal"}).text[0:10])) * ad_list[0] /10000))
    print(pViegoList)
    pViegoList.append(str(float(re.sub(r'[^0-9.,]', '',pViego.find("span",{"style" : "color: #7A6DFF; white-space:normal"}).text[0:10]))* 0 /10000))
    print(pViegoList)
    pViegoList.append(str(float(re.sub(r'[^0-9.,]', '',pViego.find("span",{"style" : "color:orangered; white-space:normal"}).text[0:10]))* bonus_as_list[0] /10000))
    print(pViegoList)
    
    wViegoCD = wViego.find("div",{"class" : "pi-data-value pi-font"}).text # W cd
    
    def somme(liste):
        _somme = 0
        for i in range(len(pViegoList)):
            _somme = _somme + float(pViegoList[i])
        return _somme
     
    print(somme(pViegoList))
    pViegoList.append(0) 
    pViegoList.append(0) 
    pViegoList.append(0) 
    pViegoList.append(0) 
    pViegoList.append(0) 
    pViegoList.append(0) 
    pViegoList.append(0) 
    pViegoList.append(0)
    pViegoList.append(0) 
    pViegoList.append(0) 
    pViegoList.append(0) 
    pViegoList.append(0) 
    pViegoList.append(0) 
    pViegoList.append(0) 
    
except AttributeError:
    print("error")

wViegoRatio = str(int(re.sub(r'[^0-9.,]','',re.search(r'\(([^)]+)\)', wViego.find("dd").text).group(1) if re.search(r'\(([^)]+)\)', wViego.find("dd").text) else None))/100) + "AP"
wViegoDmg = list(map(int,re.sub(r'\([^)]+\)', '', wViego.find("dd").text).split("/")))
print(wViegoDmg)
print(wViegoCD)
print(wViegoRatio)

['0.02']
['0.02', '0.01425']
['0.02', '0.01425', '0.0']
['0.02', '0.01425', '0.0', '0.0']
0.03425
[80, 135, 190, 245, 300]
8
1.0AP


In [188]:
# q w e - q - q - r - q - e -q -e -r -e  -w  -e- w -r -w-w

In [189]:

def find_element_by_label(label, class_name):
    if html is None:
        return None

    balise_h3 = html.find('h3', class_='pi-data-label {}'.format(class_name), string=label)

    if balise_h3:
        balise_suivante = balise_h3.find_next_sibling()

        if balise_suivante:
            return balise_suivante.text.strip()


def get_cooldown_value():
    return find_element_by_label('COOLDOWN:', 'pi-secondary-font')

def Bonus_Damage():
    stats_div = html.find('div', {'style': 'padding: 0 7px'})

    if stats_div:
        stats_dl_list = stats_div.find_all('dl', class_='skill-tabs')
        stats = {}

        for stats_dl in stats_dl_list:
            dt = stats_dl.find('dt')
            dd = stats_dl.find('dd')

            if dt and dd:
                label = dt.text.strip().replace(':', '')
                value = dd.text.strip()
                stats[label] = value 
        return stats

    return None

def format_to_list(chaine):
    nombres = []
    nombre_temporaire = ""
    est_dans_nombre = False

    for caractere in chaine:
        if caractere.isdigit() or caractere == '.':
            nombre_temporaire += caractere
            est_dans_nombre = True
        elif est_dans_nombre:
            nombres.append(float(nombre_temporaire))
            nombre_temporaire = ""
            est_dans_nombre = False

    if est_dans_nombre:
        nombres.append(float(nombre_temporaire))

    return nombres

'''
extrait le text sans balise
'''
def extract_text_without_tags(element):
    """
    Extract text content from HTML without any tags.
    """
    result = []
    for item in element.children:
        if isinstance(item, str):
            if item != "\n":
                result.append(item)
        else:
            result += html.extract_text_without_tags(item)

    return result

def get_divs_above_img(img_alt_text):
    img_tag = html.find('img', alt=img_alt_text)

    if img_tag:
        parent_div = img_tag.find_parent('div')
        divs_above = parent_div.find_all_previous('div', limit=1)

        return divs_above

    return None
def get_orange_and_blue_text(divs_above_img):
    # color #7A6DFF bleu et orange
    orange_and_blue_text = []

    for div in divs_above_img:
        orange_text = div.find_all('span', style="color:orange; white-space:normal",limit=2)
        blue_text = div.find_all('span', style="color: #7A6DFF; white-space:normal")

        for text in orange_text:
            orange_and_blue_text.append(text.get_text(strip=True))

        for text in blue_text:
            orange_and_blue_text.append(text.get_text(strip=True))

    return orange_and_blue_text

def find_physical_damage_table():
    physical_damage_span = html.find('span', {'class': 'template_lc'}, string='Physical Damage:')
    if physical_damage_span:
        table = physical_damage_span.find_previous('table')
        dd_elements = table.find('dd', class_='')
        dd_texts = [dd.get_text(strip=True) for dd in dd_elements]

    return dd_texts

def find_physical_damage_span():
    return html.find('span', {'class': 'template_lc'}, string='Physical Damage:')

# ancien main
cooldown_value = get_cooldown_value()
cooldown_value = format_to_list(cooldown_value)

print("Cooldown\n")
print(cooldown_value)
print("\n")
print("Bonus Damage Max/Min")
print(Bonus_Damage())
print("\n")
print("bonus damage 2")
img_alt_text = 'Blade of the Ruined King'
divs_above_img = get_divs_above_img(img_alt_text)
if divs_above_img:
    orange_and_blue_text = get_orange_and_blue_text(divs_above_img)
    print(orange_and_blue_text)
physical_damage_table = find_physical_damage_table()
if physical_damage_table:
    print("\nPhysical Damage Table:")
print(physical_damage_table)



Cooldown

[5.0, 4.5, 4.0, 3.5, 3.0]


Bonus Damage Max/Min
{'Bonus Physical Damage': "2 / 3 / 4 / 5 / 6% of target's current health", 'Minimum Bonus Damage': '10 / 15 / 20 / 25 / 30'}


bonus damage 2
['20% AD', '35% (+7%) AD', '(+ 15% AP)', '(+ 26.25% (+5.25%) AP)']

Physical Damage Table:
['15 / 30 / 45 / 60 / 75', '(+ 70% AD)', '', '×', '(1 + (100%\xa0+0%) critical strike chance)']


## Ultime Viego  

Ultime Cooldown 

In [190]:
soup = BeautifulSoup(response.content, 'html.parser')

ultime_cooldown = soup.find("div", {'class': 'skill skill_r'})
if ultime_cooldown:
    ultime_cooldown_value = ultime_cooldown.find('div', {'class': 'pi-data-value pi-font'})
    if ultime_cooldown_value:
        ultime_cooldown_text = ultime_cooldown_value.text
        print('COOLDOWN:', ultime_cooldown_text)
    else:
        print('Element "pi-data-value pi-font" non trouvé dans skill_r')
else:
    print('Element "skill skill_r" non trouvé dans la page')

COOLDOWN: 120 / 100 / 80


Base attack during ultimate

In [191]:
soup = BeautifulSoup(response.content, 'html.parser')
lr = soup.find("div", {'class' : 'skill skill_r'})
lr = lr.find('span', {'class' : 'pp-tooltip'})['data-displayformula']

Physical Damage

In [192]:
soup = BeautifulSoup(response.content, 'html.parser')
pdamage = soup.find("div", {'class': 'skill skill_r'})
pdamage = pdamage.find('div', {'style': 'grid-column-end: span 2; display:contents'})
pdamage = pdamage.find('span', {'style': 'color: #1F995C; white-space:normal'})

Clean data entries

In [193]:
#Divide string using (
before_parenthese = re.match(r'^(.*?)\(', pdamage.text).group(1) if re.match(r'^(.*?)\(', pdamage.text) else None
after_parenthese = re.search(r'\(.*\)', pdamage.text).group(0) if re.search(r'\(.*\)', pdamage.text) else None

#Delete % in the first string, and split
before_parenthese_without= re.sub(r'%', '', before_parenthese)
R_missing_health_dmg = before_parenthese_without.split(" / ")
R_missing_health_dmg = [int(x)/100 for x in R_missing_health_dmg]
R_missing_health_dmg = [float(x) for x in R_missing_health_dmg]

#Delete % in the second string and add it to another col
### after_parenthese = re.sub(r'%','',after_parenthese)

Stat board

In [194]:
UltimeStat = pd.DataFrame(index=range(1,4))
UltimeStat['Ultimate_cooldown'] = ultime_cooldown_text.split(" / ")
UltimeStat['R_Physical_missing_health_damages'] = R_missing_health_dmg
UltimeStat['R_Physical_missing_health_damages'] = UltimeStat['R_Physical_missing_health_damages']
UltimeStat['R_missing_health_ratio'] = str(int(re.sub(r'[^0-9.,]','',after_parenthese[0:9]))/10000) + "BONUS_AD"
UltimeStat['R_auto_dmg'] = lr
UltimeStat

,Ultimate_cooldown,R_Physical_missing_health_damages,R_missing_health_ratio,R_auto_dmg
1,120,0.12,0.0005BONUS_AD,120 x (1 + critical strike chance)
2,100,0.16,0.0005BONUS_AD,120 x (1 + critical strike chance)
3,80,0.20,0.0005BONUS_AD,120 x (1 + critical strike chance)


In [195]:
%run "../class_champion.ipynb"

In [196]:
viegoStat['rCD'] = 0
viegoStat['rMissingHealthDmg'] = 0
viegoStat['rAutoDmg'] = 0
for i in range (6,11):
    viegoStat['rCD'][i] = UltimeStat['Ultimate_cooldown'][1]
    viegoStat['rMissingHealthDmg'][i] = str(UltimeStat['R_Physical_missing_health_damages'][1])+ "+" + UltimeStat['R_missing_health_ratio'][1] + "TARGET_MISSING_HP"
for i in range (11,16):
    viegoStat['rCD'][i] = UltimeStat['Ultimate_cooldown'][2]
    viegoStat['rMissingHealthDmg'][i] = str(UltimeStat['R_Physical_missing_health_damages'][2])+ "+" + UltimeStat['R_missing_health_ratio'][2] + "TARGET_MISSING_HP"

for i in range  (16,19):
    viegoStat['rCD'][i] = UltimeStat['Ultimate_cooldown'][3]
    viegoStat['rMissingHealthDmg'][i] = str(UltimeStat['R_Physical_missing_health_damages'][3])+ "+" + UltimeStat['R_missing_health_ratio'][3] + "TARGET_MISSING_HP"

viegoStat['passive'] = "+".join(pViegoList)
viegoStat['wCD'] = wViegoCD

#Viego Wdmg + ratio
#qdmg + ratio
#qcd
#r cd
#rdmg + ratio (autre code)

viegoBase = Champion("viego",viegoStat)

C:\Users\sever\AppData\Local\Temp\ipykernel_4428\726276855.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  viegoStat['rCD'][i] = UltimeStat['Ultimate_cooldown'][1]
C:\Users\sever\AppData\Local\Temp\ipykernel_4428\726276855.py:5: SettingW

TypeError: sequence item 4: expected str instance, int found

In [ ]:
print(viegoBase)